<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import zipfile

# ===============================
# 1. Zip 파일 경로 지정
# ===============================
zip_path = "/content/sample_data/EdNet-KT3.zip"

# ===============================
# 2. 압축 파일 안에 있는 파일 목록 확인
# ===============================
with zipfile.ZipFile(zip_path, 'r') as z:
    file_list = z.namelist()
    print("압축파일 안에 있는 파일들:")
    print(file_list)

# ===============================
# 3. CSV 파일 선택 (예: KT1.csv)
# ===============================
csv_name = file_list[0]   # 첫 번째 파일 사용 (필요하면 바꾸세요)

# ===============================
# 4. CSV 읽기
# ===============================
with zipfile.ZipFile(zip_path) as z:
    with z.open(csv_name) as f:
        df = pd.read_csv(f)

# ===============================
# 5. 앞 5개 열만 출력
# ===============================
print("전체 열 이름:", df.columns.tolist())
print("\n앞 5개 열 데이터:")
print(df.iloc[:, :5].head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/EdNet-KT3.zip'